# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [4]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 1.067045     | 4146407.600631     | 258679.804477                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [6]:
example_weight_summary = example_model.coefficients
print(example_weight_summary)

+-------------+-------+--------------------+--------------------+
|     name    | index |       value        |       stderr       |
+-------------+-------+--------------------+--------------------+
| (intercept) |  None |  87910.0724923996  | 7873.3381434016665 |
| sqft_living |  None |  315.403440552103  | 3.4557003258547465 |
|   bedrooms  |  None | -65080.21555282863 | 2717.4568544207114 |
|  bathrooms  |  None | 6944.020192635838  | 3923.114931441497  |
+-------------+-------+--------------------+--------------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [10]:
example_predictions = example_model.predict(train_data)
print(example_predictions[0]) # should be 271789.505878
print(train_data[0]) # should be 271789.505878


271789.505878031
{'id': '7129300520', 'date': datetime.datetime(2014, 10, 13, 0, 0, tzinfo=GMT +0.0), 'price': 221900.0, 'bedrooms': 3.0, 'bathrooms': 1.0, 'sqft_living': 1180.0, 'sqft_lot': 5650.0, 'floors': 1.0, 'waterfront': 0, 'view': 0, 'condition': 3, 'grade': 7.0, 'sqft_above': 1180.0, 'sqft_basement': 0.0, 'yr_built': 1955.0, 'yr_renovated': 0.0, 'zipcode': '98178', 'lat': 47.51123398, 'long': -122.25677536, 'sqft_living15': 1340.0, 'sqft_lot15': 5650.0}


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [11]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)

    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    diff = predictions - outcome
    
    # square the residuals and add them up
    RSS = (diff * diff).sum()

    return(RSS)

Test your function by computing the RSS on TEST data for the example model:

In [13]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print(rss_example_train) # should be 2.7376153833e+14

273761538330193.3


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [14]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [15]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [17]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']
print(train_data[0])

{'id': '7129300520', 'date': datetime.datetime(2014, 10, 13, 0, 0, tzinfo=GMT +0.0), 'price': 221900.0, 'bedrooms': 3.0, 'bathrooms': 1.0, 'sqft_living': 1180.0, 'sqft_lot': 5650.0, 'floors': 1.0, 'waterfront': 0, 'view': 0, 'condition': 3, 'grade': 7.0, 'sqft_above': 1180.0, 'sqft_basement': 0.0, 'yr_built': 1955.0, 'yr_renovated': 0.0, 'zipcode': '98178', 'lat': 47.51123398, 'long': -122.25677536, 'sqft_living15': 1340.0, 'sqft_lot15': 5650.0, 'bedrooms_squared': 9.0, 'bed_bath_rooms': 3.0, 'log_sqft_living': 7.07326971745971, 'lat_plus_long': -74.74554138}


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [20]:
print(test_data['bedrooms_squared'].mean())
print(test_data['bed_bath_rooms'].mean())
print(test_data['log_sqft_living'].mean())
print(test_data['lat_plus_long'].mean())

12.4466777015843
7.503901631591395
7.550274679645935
-74.6533349721731


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [26]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']
model_mine_features =  ['bed_bath_rooms', 'bedrooms_squared', 'log_sqft_living']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [27]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = turicreate.linear_regression.create(train_data, target = 'price', features = model_1_features, validation_set = None)
model_2 = turicreate.linear_regression.create(train_data, target = 'price', features = model_2_features, validation_set = None)
model_3 = turicreate.linear_regression.create(train_data, target = 'price', features = model_3_features, validation_set = None)
model_mine = turicreate.linear_regression.create(train_data, target = 'price', features = model_mine_features, validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.062290     | 5531660.038293     | 285904.368081                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [28]:
# Examine/extract each model's coefficients:
print(model_1.coefficients)
print(model_2.coefficients)
print(model_3.coefficients)
print(model_mine.coefficients)

+-------------+-------+--------------------+--------------------+
|     name    | index |       value        |       stderr       |
+-------------+-------+--------------------+--------------------+
| (intercept) |  None | -56140675.74114406 | 1649985.4204707814 |
| sqft_living |  None | 310.2633257769218  |  3.1888296040769   |
|   bedrooms  |  None | -59577.11606759681 | 2487.279773224387  |
|  bathrooms  |  None | 13811.840541652673 | 3593.542132970472  |
|     lat     |  None | 629865.7894714824  | 13120.71003255109  |
|     long    |  None | -214790.2851647091 | 13284.285161712167 |
+-------------+-------+--------------------+--------------------+
[6 rows x 4 columns]

+----------------+-------+---------------------+--------------------+
|      name      | index |        value        |       stderr       |
+----------------+-------+---------------------+--------------------+
|  (intercept)   |  None | -54410676.111841485 | 1650405.1656093246 |
|  sqft_living   |  None |  304.449298

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**
A: positive

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**
A: negative

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [29]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
print(get_residual_sum_of_squares(model_1, train_data, train_data['price']))
print(get_residual_sum_of_squares(model_2, train_data, train_data['price']))
print(get_residual_sum_of_squares(model_3, train_data, train_data['price']))
print(get_residual_sum_of_squares(model_mine, train_data, train_data['price']))

971328233545434.8
961592067857509.2
905276314551640.5
1420990892841993.8


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?
A: the algorithm found a "good fit" that minimized the error on THAT data

Now compute the RSS on on TEST data for each of the three models.

In [30]:
# Compute the RSS on TESTING data for each of the three models and record the values:
print(get_residual_sum_of_squares(model_1, test_data, test_data['price']))
print(get_residual_sum_of_squares(model_2, test_data, test_data['price']))
print(get_residual_sum_of_squares(model_3, test_data, test_data['price']))
print(get_residual_sum_of_squares(model_mine, test_data, test_data['price']))

226568089093160.72
224368799993971.28
251829318963161.9
517977925542205.94


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

In [31]:
model 2

+----------------+-------+---------------------+--------------------+
|      name      | index |        value        |       stderr       |
+----------------+-------+---------------------+--------------------+
|  (intercept)   |  None | -54410676.111841485 | 1650405.1656093246 |
|  sqft_living   |  None |  304.44929805557916 | 3.202175356371528  |
|    bedrooms    |  None |  -116366.0432294497 | 4805.549665487789  |
|   bathrooms    |  None |  -77972.33051299027 | 7565.059910961288  |
|      lat       |  None |  625433.8349398108  | 13058.35309747484  |
|      long      |  None | -203958.60293746984 | 13268.128372152305 |
| bed_bath_rooms |  None |  26961.624908952617 | 1956.365615560501  |
+----------------+-------+---------------------+--------------------+

SyntaxError: invalid syntax (<ipython-input-31-5d7bc19643da>, line 1)